In [1]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Cake
# 1. The cake is in the kitchen
# 2. The cake is delicious
# 3. The cake is not eaten
# 4. The cake is sweet
# 5. The cake is not sweet
# 6. The cake is not delicious
# 7. The cake is eaten
# 8. The cake is in the stomach

def eat():
    return "eat_cake"

def bake():
    return "bake_cake"

stmap = { 
        Strips(eat(), { "cake": True}, {"cake": False}),
        Strips(bake(), { "cake": False}, {"cake": True})
    }

# all possible actions
feature_domain_dict = { "cake": { True, False} }
strips_domain = STRIPS_domain(feature_domain_dict, stmap)
cake_problem = Planning_problem(strips_domain, { "cake": False}, { "cake": True})
SearcherMPP(Forward_STRIPS(cake_problem)).search()


Solution: {'cake': False}
   --bake_cake--> {'cake': True} (cost: 1)
 2 paths have been expanded and 0 paths remain in the frontier


{'cake': False}
   --bake_cake--> {'cake': True}

In [9]:
# magic world

# Copilot stop spamming the f-cking chat

def move(x, y, nx, ny):
    return 'move_from_'+str(x)+str(y)+'_to_'+str(nx)+str(ny)

def attack(x, y):
    return "attack_dragon_at_" + str(x) + "_" + str(y)

def open():
    pass

def collectFire(x, y):
    return "collect_fire_at_" + str(x) + "_" + str(y)

def collectEarth(x, y):
    return "collect_earth_at_" + str(x) + "_" + str(y)

def buildFireball():
    return "build_fireball"

# nxn grid with player, castle, walls, and dragons
# End goal is kill the dragon and react the castle
# To kill the dragon, player has to be at dragon coordinates and has the fireball
# To build a fireball, the player needs k earth and l fire
# To attack, the player has to be at the same position as a dragon
# Player can open a chest but idk why - skip for now

def initWorld(n, dragon_coords, fire_coords, earth_coords):
    # nxn grid
    feature_domain_dict = { 
        "player": [(i, j) for i in range(n) for j in range(n)], 
        "dragon": (True, False),
        "fire": (True, False),
        "earth": (True, False),
        "fireball": (True, False), 
    }

    # all possible player movements
    stmap = {
        Strips(move(x, y, x + dx, y + dy), { "player": (x, y)}, { "player": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(n) 
            for y in range(n)
            if x + dx in range(n) and y + dy in range(n)
    }

    # collecting fire
    stmap.update({
        Strips(collectFire(fire_coords[0], fire_coords[1]), { "player": fire_coords, "fire": False}, { "fire": True })
    })

    # collecting earth
    stmap.update({
        Strips(collectEarth(earth_coords[0], earth_coords[1]), { "player": earth_coords, "earth": False}, { "earth": True })
    })

    # building a fireball
    stmap.update({
        Strips(buildFireball(), { "fire": True, "earth": True, "fireball": False}, { "fireball": True, "fire": False, "earth": False})
    })

    # attacking a dragon
    stmap.update({
       Strips(attack(dragon_coords[0], dragon_coords[1]), { "player": dragon_coords, "dragon": True, "fireball": True}, { "dragon": False, "fireball": False }) 
    })

    return STRIPS_domain(feature_domain_dict, stmap)

dragon = (1, 3)
fire = (2, 2)
earth = (3, 3)
world1 = initWorld(5, dragon, fire, earth)
player_coords = (1, 1)
castle = (4, 4)

problem = Planning_problem(
    world1, 
    { "player": player_coords, "dragon": True, "fire": False, "earth": False, "fireball": False}, 
    { "player": castle, "dragon": False}
)

sol = SearcherMPP(Forward_STRIPS(problem)).search()
print(sol)

Solution: {'player': (1, 1), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --move_from_11_to_21--> {'player': (2, 1), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --move_from_21_to_22--> {'player': (2, 2), 'dragon': True, 'fire': False, 'earth': False, 'fireball': False}
   --collect_fire_at_2_2--> {'player': (2, 2), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --move_from_32_to_33--> {'player': (3, 3), 'dragon': True, 'fire': True, 'earth': False, 'fireball': False}
   --collect_earth_at_3_3--> {'player': (3, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --move_from_33_to_23--> {'player': (2, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --move_from_23_to_13--> {'player': (1, 3), 'dragon': True, 'fire': True, 'earth': True, 'fireball': False}
   --build_fire